In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.5
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
shuffled_sample = shuffled_words.sample(n=20)
shuffled_sample

,Français,Hiragana,Kanji
82,salle / chambre,へや,部屋
73,enseignant,きょうし,教室
77,réserver,よやくします,予約します
155,réponse à yoroshiku,こちらこそよろしくおねがいします,こちらこそよろしくお願いします
12,étudiant,がくせい,学生
50,changer (de train),のりかえます,乗り換えます
6,université,だいがく,大学
30,espèces / argent liquide,げんきん,現金
121,passer [un coup de téléphone],かけます,NaN
84,donner / offrir,あげます,NaN


In [8]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Comment dire espèces / argent liquide en hiragana ?げんきん
Et en kanji ?現金
Correct !
Bonnes réponses : 1/20
Comment dire changer (de train) en hiragana ?のりかえ
Incorrect ! La bonne réponse est : のりかえます
Bonnes réponses : 1/20
Comment dire (mon) mari en hiragana ?おっと
Incorrect ! La bonne réponse est : しゅじん
Bonnes réponses : 1/20
Comment dire mairie en hiragana ?主人
Incorrect ! La bonne réponse est : しやくしょ
Bonnes réponses : 1/20
Comment dire enseignant en hiragana ?きょうし
Et en kanji ?教師
Incorrect ! La bonne réponse est : 教室
Bonnes réponses : 1/20
Comment dire café / salon de thé en hiragana ?かふぇ
Incorrect ! La bonne réponse est : きっさてん
Bonnes réponses : 1/20
Comment dire réponse à yoroshiku en hiragana ?
Incorrect ! La bonne réponse est : こちらこそよろしくおねがいします
Bonnes réponses : 1/20
Comment dire perdre en hiragana ?うしなう
Incorrect ! La bonne réponse est : なくします
Bonnes réponses : 1/20
Comment dire salle de cours en hiragana ?きょうしつ
Et en kanji ?教室
Correct !
Bonnes réponses : 2/20
Comment dire feu de rou

In [9]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Que veut dire ce kanji : 乗り換えます?changer de train
Correct ! C'était bien changer (de train)
Bonnes réponses : 1/20
Que veut dire ce kanji : 部屋?le chambre
Correct ! C'était bien salle / chambre
Bonnes réponses : 2/20
Que veut dire ce kanji : 教室?la salle de cours
Incorrecte ! La bonne réponse était : enseignant
Bonnes réponses : 2/20
Que veut dire ce kanji : こちらこそよろしくお願いします?
Correct ! C'était bien réponse à yoroshiku
Bonnes réponses : 3/20
Que veut dire ce kanji : 学生?etudiant
Correct ! C'était bien étudiant
Bonnes réponses : 4/20
Que veut dire ce kanji : 予約します?réserver
Correct ! C'était bien réserver
Bonnes réponses : 5/20
Que veut dire ce kanji : 休みます?congé
Correct ! C'était bien se reposer / prendre un congé
Bonnes réponses : 6/20
Que veut dire ce kanji : 返します?rendre
Correct ! C'était bien rendre / rembourser
Bonnes réponses : 7/20
Que veut dire ce kanji : 市役所?mairie
Correct ! C'était bien mairie
Bonnes réponses : 8/20
Que veut dire ce kanji : 教室?la salle de cours
Correct ! C'était bien